# Roads Import

In order to have the Django environment vars, settings, and app load correctly,
run this notebook from the Roads repo using: 

```bash
./manage.py shell_plus --notebook
```

Ensure that the notebook kernel is set to "Django Shell-Plus", not "Python3".

Make a symlink or copy from the shared roads directory to `ngis`.
This sits in the root dir of wherever you cloned the Roads repo.

## Special Handling


### LinestringZM: Four Dimensions

The `RRMPIS` file has four dimensions (x,y,z,m where m is for instance "time this point was taken on GPS"), and fails to load. The `Roads` table has three dimensions. Use ogr2ogr to drop the `M` (4th) dimension. Note that ogr2ogr arguments are the destination file before the source file.

For example: `ogr2ogr -dim XYZ 2d_RRMPIS_2014.shp RRMPIS_2014.shp`

### Bad Geometries

When a "bad" geometry is encountered (which so far are some features in RRMPIS_2014 with no geometry data) it would rais an error as soon as one uses `feature.geom`. This is excepted out at the start of read_shape_file.


In [145]:
from django.contrib.gis.gdal import DataSource, GDALException
from django.contrib.gis.geos import GEOSGeometry, LineString, MultiLineString
from assets.models import Road, SourceNationalRoad, SourceMunicipalRoad, SourceRrmpis, SourceR4D
from pathlib import Path

source_dir = Path('.') / '..' / '..' / 'ngis'


In [146]:
national_roads_shp = '%s' % (source_dir / 'National_Road.shp').absolute()
print(national_roads_shp)
municipal_roads_shp = '%s' % (source_dir / 'Municipal_Road.shp')
rural_roads_r4d_shp = '%s' % (source_dir / 'Rural_Road_R4D_Timor_Leste.shp')

# See notes above. This is derived from RRMPIS_2014.
rural_roads_rrmpis_shp = '%s' % (source_dir / '2d_RRMPIS_2014.shp')

/home/josh/github/catalpainternational/roads/IRMIS/../../ngis/National_Road.shp


In [147]:
def read_shape_file(shp_path, source_model, unique_keys):
    shp_file = DataSource(shp_path)
    for feature in shp_file[0]:
        database_srid = Road._meta.fields[1].srid
        
        try:
            # Try accessing the feature geometry. If this fails it is usually because of an empty geometry. It may also
            # be because the `Roads` table is XYZ coordinates and your input file (such as RRMPIS_2014) is in XYZM. 
            feature.geom
        except GDALException:
            print('!!! Error in geometry')
            continue

        srid = feature.geom.srid

        print('examining feature ', feature.fid, feature[unique_keys[0]], ' original srid=', srid)
        
        # try to get the source properties
        try:
            values = {key: feature[key] for key in unique_keys}
            properties = source_model.objects.get(**values)
        except source_model.MultipleObjectsReturned as ex:
            properties = source_model.objects.filter(**values).first()
            print('more than one match for ', ['{}={}'.format(key, val) for key, val in values.items()], 'using first found')
        except source_model.DoesNotExist as ex:
            print('empty row')
            continue

        # turn line strigns into multilinestrings
        try:
            if isinstance(feature.geom.geos, LineString):
                print('LineString - converting to MultiLineString')
                multi_line_string = MultiLineString(feature.geom.geos)
            elif isinstance(feature.geom.geos, MultiLineString):
                print('MultiLineString - using as is')
                multi_line_string = feature.geom.geos
            else:
                print('!!!!! unhandled geometry type', feature.geom.geos)
        except GDALException as ex:
            print(dir(feature))
            print(feature[0])
            print('!!!!!!!', ex)
            break
            
        # create Road
        road_geometry = GEOSGeometry(multi_line_string, srid=srid)
        if srid != database_srid:
            road_geometry.transform(database_srid)
        road = Road.objects.create(properties=properties, geom=road_geometry.wkt)
        print('created', road)
        print('----------')

In [149]:
Road.objects.all().delete()
    
read_shape_file(national_roads_shp, SourceNationalRoad, ['name', 'subcode', 'code'])
read_shape_file(municipal_roads_shp, SourceMunicipalRoad, ['name', 'descriptio'])
read_shape_file(rural_roads_r4d_shp, SourceR4D, ['road_lin_1', 'id'])
read_shape_file(rural_roads_rrmpis_shp, SourceRrmpis, ['rd_id', 'rdcode_cn', 'sheet_ref'])